In [113]:


from dotenv import load_dotenv
load_dotenv()

import os
import streamlit as st



from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

llm.invoke("LLM ready").content




from langchain_community.tools import DuckDuckGoSearchRun

ddg = DuckDuckGoSearchRun()

ddg_result = ddg.invoke(
    "TATA company overview competitors market position"
)




from tavily import TavilyClient

tavily = TavilyClient(api_key="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")

tavily_queries = {
    "recent_news": "TATA recent news press release",
    "earnings": "TATA earnings announcement",
    "future_plans": "TATA future plans management statement",
    "stock_news": "TATA stock news market reaction"
}

tavily_results = {}

for key, query in tavily_queries.items():
    tavily_results[key] = tavily.search(
        query=query,
        search_depth="advanced"
    )



research_state = {
    "company": "TATA",
    "duckduckgo": ddg_result,
    "tavily": tavily_results
}


from pydantic import BaseModel
from typing import List

class DetailedCompanyReport(BaseModel):
    executive_summary: str
    company_overview: str
    recent_developments: List[str]
    future_plans: List[str]
    stock_context: str
    risks_and_challenges: List[str]
    sources_used: List[str]


from langchain.prompts import ChatPromptTemplate

structured_llm = llm.with_structured_output(DetailedCompanyReport)

analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are an evidence-based market research analyst.

Rules:
- Use ONLY provided information
- Do NOT invent facts or numbers
- No stock prediction
- Future plans must be explicitly announced
- If info is missing, state clearly
"""),
    ("human", """
DuckDuckGo Overview:
{ddg}

Recent News:
{recent_news}

Earnings:
{earnings}

Future Plans:
{future_plans}

Stock News:
{stock_news}

Generate a detailed, factual company report.
""")
])

analysis_messages = analysis_prompt.format_messages(
    ddg=research_state["duckduckgo"],
    recent_news=research_state["tavily"]["recent_news"],
    earnings=research_state["tavily"]["earnings"],
    future_plans=research_state["tavily"]["future_plans"],
    stock_news=research_state["tavily"]["stock_news"]
)

final_report = structured_llm.invoke(analysis_messages)

report_dict = final_report.model_dump()


import sqlite3
import json
from datetime import datetime

conn = sqlite3.connect("company_reports.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS reports (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company TEXT,
    report_json TEXT,
    created_at TEXT
)
""")

conn.commit()

cursor.execute("""
INSERT INTO reports (company, report_json, created_at)
VALUES (?, ?, ?)
""", (
    research_state["company"],
    json.dumps(report_dict),
    datetime.utcnow().isoformat()
))

conn.commit()




cursor.execute("""
SELECT company, report_json, created_at
FROM reports
ORDER BY created_at DESC
LIMIT 1
""")

row = cursor.fetchone()

company_name = row[0]
stored_report = json.loads(row[1])
created_at = row[2]



from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

file_name = f"{company_name}_Evidence_Based_Report.pdf"

pdf = SimpleDocTemplate(file_name, pagesize=A4)
styles = getSampleStyleSheet()
content = []

def add_section(title, data):
    content.append(Paragraph(f"<b>{title}</b>", styles["Heading2"]))
    content.append(Spacer(1, 10))
    if isinstance(data, list):
        for item in data:
            content.append(Paragraph(f"- {item}", styles["Normal"]))
    else:
        content.append(Paragraph(data, styles["Normal"]))
    content.append(Spacer(1, 15))

content.append(Paragraph(
    f"<b>{company_name} – Evidence-Based Market Research Report</b>",
    styles["Title"]
))
content.append(Spacer(1, 20))

content.append(Paragraph(f"Generated on: {created_at}", styles["Normal"]))
content.append(Spacer(1, 20))

add_section("Executive Summary", stored_report["executive_summary"])
add_section("Company Overview", stored_report["company_overview"])
add_section("Recent Developments", stored_report["recent_developments"])
add_section("Future Plans (Announced)", stored_report["future_plans"])
add_section("Stock Context", stored_report["stock_context"])
add_section("Risks & Challenges", stored_report["risks_and_challenges"])
add_section("Sources", stored_report["sources_used"])

pdf.build(content)

file_name


st.set_page_config(page_title="AI Market Research", layout="wide")

st.title("AI Market Research System")

tab1, tab2 = st.tabs(["Generate Report", "Profile & History"])



KeyboardInterrupt: 

In [112]:
import streamlit as st
import sqlite3
import json
import os
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from tavily import TavilyClient

from pydantic import BaseModel
from typing import List

from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet



# -------------------------
# DATABASE
# -------------------------

conn = sqlite3.connect("company_reports.db", check_same_thread=False)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS reports (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_email TEXT,
    company TEXT,
    pdf_path TEXT,
    created_at TEXT
)
""")

conn.commit()



# -------------------------
# LLM
# -------------------------

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)



# -------------------------
# REPORT GENERATION LOGIC
# -------------------------

def generate_report(company):

    ddg = DuckDuckGoSearchRun()
    ddg_result = ddg.invoke(f"{company} company overview")

    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    news = tavily.search(f"{company} recent news", search_depth="advanced")

    content = f"""
Company: {company}

Overview:
{ddg_result}

Recent News:
{news}
"""

    file_name = f"{company}_{datetime.utcnow().isoformat()}.pdf"

    pdf = SimpleDocTemplate(file_name, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph(f"{company} Research Report", styles["Title"]))
    story.append(Spacer(1, 12))
    story.append(Paragraph(content, styles["Normal"]))

    pdf.build(story)

    return file_name



# -------------------------
# STREAMLIT UI
# -------------------------

st.set_page_config(page_title="AI Market Research", layout="wide")

st.title("AI Market Research System")

tab1, tab2 = st.tabs(["Generate Report", "Profile & History"])



# -------- TAB 1 --------

with tab1:
    st.subheader("Generate Company Report")

    email = st.text_input("Your Email")
    company = st.text_input("Company Name")

    if st.button("Generate Report"):

        if not email or not company:
            st.error("Please fill all fields")
        else:
            st.info("Generating report...")

            pdf_path = generate_report(company)

            cursor.execute("""
            INSERT INTO reports (user_email, company, pdf_path, created_at)
            VALUES (?, ?, ?, ?)
            """, (
                email,
                company,
                pdf_path,
                datetime.utcnow().isoformat()
            ))

            conn.commit()

            st.success("Report generated!")

            with open(pdf_path, "rb") as f:
                st.download_button(
                    "Download PDF",
                    f,
                    file_name=pdf_path
                )



# -------- TAB 2 --------

with tab2:
    st.subheader("Your Report History")

    email_lookup = st.text_input("Enter your email to view history")

    if st.button("Load History"):
        cursor.execute("""
        SELECT company, pdf_path, created_at
        FROM reports
        WHERE user_email = ?
        ORDER BY created_at DESC
        """, (email_lookup,))

        rows = cursor.fetchall()

        if not rows:
            st.warning("No reports found")
        else:
            for company, pdf, date in rows:
                st.write(f"**{company}** — {date}")
                with open(pdf, "rb") as f:
                    st.download_button(
                        f"Download {company}",
                        f,
                        file_name=pdf
                    )


2026-01-13 16:09:17.255 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 16:09:17.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar